In [1]:
import pandas as pd
import numpy as np
from time import time
import re
from tqdm import tqdm
from sklearn.metrics import jaccard_score
import missingno as msno
from google.cloud import bigquery


In [2]:
bqclient = bigquery.Client(project='charged-genre-350106')

### 제주패스 차종 

In [3]:
query_rentcar= """
SELECT
  product_name,
  car_type
FROM 
  `charged-genre-350106.bcml.bcml_rentcar`
"""

In [4]:
jjps= bqclient.query(query_rentcar).to_dataframe()

In [5]:
#null값 제거
jjps=jjps.dropna(axis=0)

#중복 자동차 이름 
jjps=jjps.drop_duplicates(['product_name'])

#리인덱싱(중간에 빈 인덱스)
jjps=jjps.reset_index()

#괄호 및 괄호안의 문자 제거 
for i in tqdm(range(len(jjps))):
    jjps['product_name'][i]=re.sub('\(.*\)|\s-\s.*','',jjps['product_name'][i])

#필요 데이터만 남기고
jjps=jjps[['product_name','car_type']]

#컬럼명 수정
jjps.columns=['car_name','car_type']

  0%|          | 0/392 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
100%|██████████| 392/392 [00:00<00:00, 3728.52it/s]


In [6]:
jjps

,car_name,car_type
0,지프 레니게이드,05
1,더 뉴 모닝,01
2,카니발 4세대 9인,06
3,팰리세이드 8인,05
4,SM6,03
...,...,...
387,제네시스 EQ900,04
388,코란도 투리스모 9인,06
389,더 뉴 트랙스,05
390,YF 쏘나타,03


In [8]:
# 원본 데이터 확인
jjps.to_csv('jjps.csv', encoding='euc-kr')

In [96]:
# 차종 코드별로 통합
jjps=jjps.groupby('car_type')['car_name'].apply(' '.join).to_frame().reset_index()

In [97]:
jjps

,car_type,car_name
0,01,더 뉴 모닝 캐스퍼 인스퍼레이션 더 뉴 스파크 올 뉴 모닝 캐스퍼 더 넥스트 스파크...
1,02,엑센트 볼트 EV 미니 쿠퍼 컨버터블 엑센트 CLIO 미니 쿠퍼 컨버터블S 엑센트 ...
2,03,SM6 더 뉴 말리부 쏘나타 뉴 라이즈 벤츠 E220D 카브리올레 테슬라모델3 LF...
3,04,그랜저 IG 포드 머스탱 컨버터블 더 뉴 그랜저IG 지프 랭글러 루비콘 제네시스 ...
4,05,지프 레니게이드 팰리세이드 8인 코나 쏘렌토 4세대 7인 더 뉴 쏘렌토 7인 쏘렌...
5,06,카니발 4세대 9인 더 뉴 카니발 11인 카니발 4세대 9인 스타리아 9인 올 뉴 ...
6,07,SM3 올 뉴 아반떼CN7 토요타 프리우스 미니 쿠퍼 컨버터블 SM3 ZE K3 미...
7,08,올 뉴 니로 EV 제네시스 GV60 BMW I4 코란도 이모션
8,09,미니 쿠퍼 컨버터블JCW 미니 쿠퍼 컨버터블 BMW 420i 컨버터블 미니 쿠퍼 컨...


In [98]:
#차종별 데이터 확인
#''.join((jjps['car_name'][0]))

In [99]:
#불용어 정리 for 제주패스 차종
stop_words="더 뉴 넥스트 인스퍼레이션 올 어반 차박 캠핑 EV 컨버터블 컨버터블S E208 라이즈 카브리올레 LF DN8 1세대 3세대 1.6 터보 8세대 2세대 \
            4세대 4인 5인 6인 7인 8인 9인 11인 12인 15인 TM 하이리무진 스포츠 2.0 718 250 마스터 티 R AWD 4륜 2세대 3세대 휘 LPG 휘발유 \
            가솔린 경유 프리미엄 플래티넘 올뉴 더뉴 LE 19 2015 2016 2017 2018 2019 2020 2021 2022 만26세 9인승 AWD │ 2500 초이스 AT 2 \
            3800 The II AD G 147호1005 │105허1117 L D 24호2412 20 디젤2WD THE SUV 22 디젤 디럭스 New 1600 VGT 디럭스 105하9373\
            713하2001 138하1356 41하8080 63호0394 7인20 109호1363 렌터카 모던 30 스탠다드 202021 2WD 138하1195 43호0974 2111 NEW ALL 22 14 T 25 All new 프리미어 \
            20182020 엠디 201618 16 20162021 202122 3세대0 30 55허8203 162호1003 201718 2018더뉴 엘피지 1821 모자이카 긴출 21년휘발유 19년 1719 GTe 2세대2022년 \
            LPG2021 7인승 16T 7인승경 컨버터블자차 21년 1920LPG 1920년 럭셔리휘 1819 1819휘발유 2021년 1819경유 19년휘발유 20년휘발유 22년 휘발유 2021년휘발유 5새대가솔린2WD \
            19년LPG 아이소픽스 썬루프 Lpi 롱 레인지 퍼포먼스 4도어 롱레인지 C GLE 3008 200 GLB 5도어 제타 이모션 5세대 플레티넘 저상 휠체어 슬로프 투어 라운지 수동 마스터 네오 2008 MASTER"

In [100]:
#수정이 필요한 단어 정리
revise_word="아반떼ad 아반테AD 그랜져 그랜저 PEUGEOT 산타페 싼타페 시트로앵 시트로엥"

In [101]:
# 불용어 제거
stop_words = set(stop_words.split(' '))
word_tokens = ''.join((jjps['car_name'][0])).split()

result = [word for word in word_tokens if not word in stop_words]

print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)

불용어 제거 전 : ['더', '뉴', '모닝', '캐스퍼', '인스퍼레이션', '더', '뉴', '스파크', '올', '뉴', '모닝', '캐스퍼', '더', '넥스트', '스파크', '더', '뉴', '레이', '레이', '더', '뉴', '모닝', '모닝', '모닝', '어반', '올', '뉴', '모닝', '올', '뉴', '모닝', '더', '뉴', '레이', '더', '뉴', '레이', '더', '뉴', '스파크', '모닝', '어반', '차박', '캠핑', '레이', '스파크']
불용어 제거 후 : ['모닝', '캐스퍼', '스파크', '모닝', '캐스퍼', '스파크', '레이', '레이', '모닝', '모닝', '모닝', '모닝', '모닝', '레이', '레이', '스파크', '모닝', '레이', '스파크']


In [102]:
for i in tqdm(range(len(jjps))):
    word_tokens = ''.join((jjps['car_name'][i])).split()
    result = [word for word in word_tokens if not word in stop_words]
    jjps['car_name'][i]=result

100%|██████████| 9/9 [00:00<00:00, 2402.85it/s]


In [103]:
jjps

,car_type,car_name
0,01,"[모닝, 캐스퍼, 스파크, 모닝, 캐스퍼, 스파크, 레이, 레이, 모닝, 모닝, 모..."
1,02,"[엑센트, 볼트, 미니, 쿠퍼, 엑센트, CLIO, 미니, 쿠퍼, 엑센트, 푸조]"
2,03,"[SM6, 말리부, 쏘나타, 벤츠, E220D, 테슬라모델3, 쏘나타, 쏘나타DN8..."
3,04,"[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ..."
4,05,"[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 쏘렌토, 쏘렌토, 스포티지, 쏘렌..."
5,06,"[카니발, 카니발, 카니발, 스타리아, 카니발, 카니발, 리무진, 스타리아, 그랜드..."
6,07,"[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, SM3, ZE, K3, ..."
7,08,"[니로, 제네시스, GV60, BMW, I4, 코란도]"
8,09,"[미니, 쿠퍼, 컨버터블JCW, 미니, 쿠퍼, BMW, 420i, 미니, 쿠퍼, 컨..."


In [104]:
#중복제거 컬럼 만들고
jjps['car_noduplicate']=0

In [105]:
jjps

,car_type,car_name,car_noduplicate
0,01,"[모닝, 캐스퍼, 스파크, 모닝, 캐스퍼, 스파크, 레이, 레이, 모닝, 모닝, 모...",0
1,02,"[엑센트, 볼트, 미니, 쿠퍼, 엑센트, CLIO, 미니, 쿠퍼, 엑센트, 푸조]",0
2,03,"[SM6, 말리부, 쏘나타, 벤츠, E220D, 테슬라모델3, 쏘나타, 쏘나타DN8...",0
3,04,"[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ...",0
4,05,"[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 쏘렌토, 쏘렌토, 스포티지, 쏘렌...",0
5,06,"[카니발, 카니발, 카니발, 스타리아, 카니발, 카니발, 리무진, 스타리아, 그랜드...",0
6,07,"[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, SM3, ZE, K3, ...",0
7,08,"[니로, 제네시스, GV60, BMW, I4, 코란도]",0
8,09,"[미니, 쿠퍼, 컨버터블JCW, 미니, 쿠퍼, BMW, 420i, 미니, 쿠퍼, 컨...",0


In [106]:
#중복제거 루프
for r in range(len(jjps)):
    a=[]
    for value in jjps['car_name'][r]:
        if value not in a:
            a.append(value)
            jjps['car_noduplicate'][r]=a

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [83]:
jjps

,car_type,car_name,car_noduplicate
0,01,"[모닝, 캐스퍼, 스파크, 모닝, 캐스퍼, 스파크, 레이, 레이, 모닝, 모닝, 모...","[모닝, 캐스퍼, 스파크, 레이]"
1,02,"[엑센트, 볼트, 미니, 쿠퍼, 엑센트, CLIO, 미니, 쿠퍼, 엑센트, 푸조]","[엑센트, 볼트, 미니, 쿠퍼, CLIO, 푸조]"
2,03,"[SM6, 말리부, 쏘나타, 벤츠, E220D, 테슬라모델3, 쏘나타, 쏘나타DN8...","[SM6, 말리부, 쏘나타, 벤츠, E220D, 테슬라모델3, 쏘나타DN8, BMW..."
3,04,"[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ...","[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ..."
4,05,"[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 쏘렌토, 쏘렌토, 스포티지, 쏘렌...","[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 스포티지, 볼보, XC40, 싼타..."
5,06,"[카니발, 카니발, 카니발, 스타리아, 카니발, 카니발, 리무진, 스타리아, 그랜드...","[카니발, 스타리아, 리무진, 그랜드, 스타렉스, 쏠라티, 벤츠, 스프린터, 뉴카운..."
6,07,"[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, SM3, ZE, K3, ...","[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, ZE, K3, i30, ..."
7,08,"[니로, 제네시스, GV60, BMW, I4, 코란도]","[니로, 제네시스, GV60, BMW, I4, 코란도]"
8,09,"[미니, 쿠퍼, 컨버터블JCW, 미니, 쿠퍼, BMW, 420i, 미니, 쿠퍼, 컨...","[미니, 쿠퍼, 컨버터블JCW, BMW, 420i, 컨트리맨, 에스컬레이드, 포르쉐..."


In [84]:
#필요한 컬럼만
jjps=jjps[['car_type','car_noduplicate']]

In [85]:
#대문자로 바꾸기위한 컬럼 생성
jjps['car_name']=0

In [86]:
#소문자를 대문자로 변환
for k in range(len(jjps)):
    upper=[]
    for l in jjps['car_noduplicate'][k]:
        upper.append(l.upper())
        jjps['car_name'][k]=upper

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [87]:
jjps=jjps[['car_type','car_name']]

In [88]:
jjps.to_csv('jjps_final.csv', encoding='euc-kr')

### ERP 차종 확인

In [21]:
query_erp= """
SELECT
  model_name
FROM 
  `charged-genre-350106.bcml.bcml_erp`
"""

In [22]:
# 데이터 로드
erp = bqclient.query(query_erp).to_dataframe()

# 중복 데이터 제거
erp=erp.drop_duplicates(['model_name'])

# 리인덱싱(중간에 빠진 인덱스넘버 제거)
erp=erp.reset_index()

# 특수문자 및 괄호 및 괄호 안 문자 제거
for i in tqdm(range(len(erp))):
    erp['model_name'][i]=re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]','',erp['model_name'][i])
    erp['model_name'][i]=re.sub('\(.*\)|\s-\s.*','',erp['model_name'][i])

# 필요한 컬럼 선택
erp=erp[['model_name']]
erp

# (jjps.iloc[0].to_dict()['car_name']).split(',')
# jjps_dic=jjps.iloc[0].to_dict()
# jjps_dic

# car_type을 입력할 컬럼 생성
erp['car_type']='00'



  0%|          | 0/1203 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
100%|██████████| 1203/1203 [00:00<00:00, 1796.54it/s]


In [23]:
erp

,model_name,car_type
0,K3,00
1,K5,00
2,k3,00
3,SM3,00
4,19K5,00
...,...,...
1198,쏘나타 뉴 라이즈 L 2018 │ 24호2412,00
1199,쏘렌토 더 마스터 D 2019 │105허1117,00
1200,아반떼 AD 스포츠 G 2018 │ 147호1005,00
1201,2020 The K9 3800 플래티넘 II AT 휘발유 2020,00


In [24]:
for i in tqdm(range(len(erp))):
    word_tokens = ''.join((erp['model_name'][i])).split()
    result = [word for word in word_tokens if not word in stop_words]
    erp['model_name'][i]=result

  0%|          | 0/1203 [00:00<?, ?it/s]


NameError: name 'stop_words' is not defined

In [ ]:
erp

In [ ]:
# ''.join((erp['model_name'][1191]))

In [ ]:
#불용어 정리 for erp 차종
remove_word=['시리즈', '더뉴','뉴', '올','올뉴', '91', 'DN8', '12', '만26세', '구형1617', '4인승', '오픈카', '7인승','8인승경', '8인승', 
             '2021', '9인승', '11인승', '12인', 'LPG0', 'HG', 'ISO쥬니어', 'L', '3세대','4세대', '넥스트', '휘', '5인승경', 
             '7인승경', '9인승경', '2020','2022', '2018', 'dn8' ,'18' ,'19',  '21', '5도어', '20년', '4W','4WD', 'CN7G','더올','PG',
            'G70','G80','에어','D3','신형','202026세','10','2223년','D32020','F','부스터','526세','전기차','20년PG','11',
            '하이브리드','라이즈','럭셔리','IGPG','DN26세','년', '경유', '베리','인','더올','발유','KA4','SG2Hybrid','K디올','경유','인승','만세','AK','세대','MQ','4GK4414','15',
            '2D','더마스터','SX','하이리무진733','17','MX00CVV','고급형','에이디','IG3000PI베이직','9P20EVGT','IG30렌터카PI모던','XDRIVE','리무진9','TN그랜드','61호64','승','기본형','프리미어','장기',
            '발류','380','NX4','에스컬레이드D','하이리무진','22','23','45TSI콰트로스포트백','22','GDI38','유MQ4','8세대','더','경','가솔린','5세대D','5세대','C0카브리레','2세대','더제타',
             '해비치프로모션','35TDI쿼트로','TUCSON','15승오토','엘피지','볼드','로드스터자차','롱레지어스','02W','CN723','THENEW','30PI','SEDAN','전기','23','16','1','G',
            '다이치&조이','다이치시즌2', '26세','IX','38', ]
#'20','9','G9','D'

In [25]:
#불용어 제거
for i in tqdm(range(len(erp))):
    word_tokens = ''.join((erp['model_name'][i]))
    for j in range(len(remove_word)):
        word_tokens=word_tokens.replace(remove_word[j], "")
        erp['model_name'][i]=word_tokens

100%|██████████| 1203/1203 [00:41<00:00, 29.03it/s]


In [26]:
erp

,model_name,car_type
0,K3,00
1,K5,00
2,k3,00
3,SM3,00
4,K5,00
...,...,...
1198,쏘나타 │ 24호24,00
1199,쏘렌토 마스터 D 2 │5허,00
1200,아반떼 AD 스포츠 │ 47호5,00
1201,The K9 플래티넘 II AT,00


In [27]:
#불용어 제거 데이터 확인
erp.to_csv('erp.csv', encoding='euc-kr')

In [37]:
#소문자를 대문자로 변환
erp['model_name']=erp['model_name'].str.upper()

In [38]:
erp

,model_name,car_type
0,K3,00
1,K5,00
2,K3,00
3,K8,00
4,CT5,00
...,...,...
1197,쏘나타,00
1198,쏘렌토,00
1199,아반떼,00
1200,K9,00


### 텍스트 유사도(자카드) 기반 맵핑

In [39]:
# 자카드 유사도 함수
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [40]:
#list((erp.iloc[0].to_dict()['model_name']).lower())

In [41]:
jjps_dic=jjps.iloc[0].to_dict()

In [42]:
erp_dic=erp.iloc[14].to_dict()

In [43]:
erp_dic['model_name']

'레이'

In [44]:
a=erp_dic['model_name'].split()
a

['레이']

In [45]:
jjps_dic['car_name']

['모닝', '캐스퍼', '스파크', '레이']

In [46]:
jaccard_similarity(jjps_dic['car_name'],erp_dic['model_name'].split())

0.25

In [52]:
for i in range(len(jjps)):
    jjps_dic=jjps.iloc[i].to_dict()
    for j in range(len(erp)):
        erp_dic=erp.iloc[j].to_dict()
        if jaccard_similarity((jjps_dic['car_name']),(erp_dic['model_name'].split())) != 0:
            erp['car_type'].iloc[j]=jjps_dic['car_type']

In [48]:
for i in range(len(jjps)):
    #similarity=[]
    jjps_dic=jjps.iloc[i].to_dict()
    #print((jjps_dic['car_name']))
    #print((jjps_dic['car_type']))
    for j in range(len(erp)):
        erp_dic=erp.iloc[j].to_dict()
        #print((erp_dic['model_name']))
        #similarity.append(jaccard_similarity((jjps_dic['car_name']),(erp_dic['model_name'].split())))
        
        #if jaccard_similarity((jjps.iloc[i].to_dict()['product_name']).lower().split(','),(model_erp.iloc[j].to_dict()['model_name']).lower().split(',')) != 0:
        if jaccard_similarity((jjps_dic['car_name']),(erp_dic['model_name'].split())) != 0:
            erp['car_type'].iloc[i]=jjps_dic['car_type']
        #print(similarity)
    #print("--------------")

In [54]:
erp.to_csv('erp_added.csv', encoding='euc-kr')

In [56]:
jjps['car_name'][0]

['모닝', '캐스퍼', '스파크', '레이']

In [64]:
a = '모닝'
b = '모닝5'
if a in b:
    print('okay')
else:
    print('no')

okay


In [ ]:
p.match(

In [217]:
jjps.to_csv('jjps.csv', encoding='euc-kr')

In [170]:
jjps.iloc[0].to_dict()

{'car_type': '01',
 'car_name': ['모닝',
  '캐스퍼',
  '스파크',
  '모닝',
  '캐스퍼',
  '넥스트',
  '스파크',
  '레이',
  '레이',
  '모닝',
  '모닝',
  '모닝',
  '모닝',
  '모닝',
  '레이',
  '레이',
  '스파크',
  '모닝',
  '레이',
  '스파크']}

In [38]:
for i in range(len(jjps)):
    jjps_dic=jjps.iloc[i].to_dict()
    for j in range(len(erp)):
        erp_dic=erp.iloc[i].to_dict()
        if jaccard_similarity((jjps.iloc[i].to_dict()['car_name']).lower().split(','),(erp.iloc[j].to_dict()['model_name']).lower().split(',')) != 0:
            erp['car_type'].iloc[j]=jjps_dic['car_type']


In [44]:
erp.to_csv('erp.csv', encoding='euc-kr')

In [ ]:
'더','뉴','인스퍼레이션','올','어반','차박','캠핑'
'EV','컨버터블','컨버터블S','E208',
'라이즈','카브리올레','LF','DN8','3세대','1.6','터보','8세대','2세대','4세대','7인','8인','9인','11인','15인','TM',
'하이리무진','스포츠',

In [ ]:
'2세대','3세대',
'휘','휘발유','가솔린','경유',
'프리미엄','플래티넘','올뉴','더뉴','LE',
'19', '2016','2017','2018','2019','2020','2021','2022',
'만26세',
'9인승',
'AWD',


In [41]:
a='K53세대'

In [42]:
a.replace('3세대','')

'K5'

In [90]:
myHost = "webisfree.com"
search = "ree"
if search in myHost:
    print("Okay")
else:
    print("None")

Okay
